# Constructing
In order to process input data easly, We may want to construct the 3D representation from input data. For example, we can create a point cloud from a depth image to apply point cloud processing such as filtering and ICP. This section introduce method to construct 3D and 2.5D representations from other representations. 

The section use methods as following:
- To a point cloud
  - From triangle meshes
  - From a depth image
  - From voxels
- To voxels
  - From a point cloud
  - From triangle meshes
- To triangle meshes
  - From voxels


In [1]:
%load_ext autoreload
%autoreload 2

## To a point cloud

### From triangle meshes

This subsection intorduce a method to sample a point cloud from a triangle meshes. The subsection tutorial use following code:

In [6]:
from tutlibs.constructing import mesh_to_point
from tutlibs.io import Mesh
from tutlibs.visualization import JupyterVisualizer as jv
import inspect

In [4]:
vertices, triangles, data = Mesh.read("../data/bunny_tm.ply")
point_cloud = mesh_to_point(vertices, triangles, 2000)
obj_points = jv.point(point_cloud, point_size=0.005)
jv.display([obj_points])

Output()

This output show a point cloud sampled from triangle meshs. This method samples points randomly from faces of a mesh model.  
The implementation is as follows.

In [8]:
print(inspect.getsource(mesh_to_point))


def mesh_to_point(
    vertices: np.ndarray, triangles: np.ndarray, num_samples: int
) -> np.ndarray:
    """Sample a point cloud from a mesh data.

    Args:
        vertices: (V, 3)
        triangles: (T, 3)
        num_samples: number of samples

    Return:
        point cloud, (num_samples, 3)
    """

    triangle_vertices = gather(vertices, triangles)  # shape: (T, 3, 3)

    # select triangle meshs
    triangle_areas = (
        np.linalg.norm(
            np.cross(
                triangle_vertices[:, 0] - triangle_vertices[:, 1],
                triangle_vertices[:, 2] - triangle_vertices[:, 1],
            ),
            ord=2,
            axis=1,
        )
        / 2
    )
    triangle_choice_weights = triangle_areas / np.sum(triangle_areas)
    num_triangles = len(triangles)
    triangle_indices = np.random.choice(
        num_triangles, num_samples, p=triangle_choice_weights
    )
    triangle_vertices = triangle_vertices[triangle_indices]

    # compute points on triang

The above process is as follows:

1. Randomly select faces to sample points. Probability of selecting a face depends on each face area.
   1. The larger the area, the higher the probability.
2. Randomly sample points from each face with the barycentric coordinate system.
   1. Sample points by multiplying normalized random values and length of lines between barycenter and vertices.

### From a depth image
This subsection intorduce a method to construct a point cloud from a depth image. The subsection tutorial use following code:

In [6]:
import cv2
from tutlibs.constructing import depth_to_point
from tutlibs.visualization import JupyterVisualizer as jv

In [7]:
fx = fy = 525
cx = 319.5
cy = 239.5

depth_image = cv2.imread("../data/redwood_3dscan_depth.png", -1)
point_cloud, _ = depth_to_point(depth_image, fx, fy, cx, cy)
obj_points = jv.point(point_cloud)
jv.display([obj_points])

Output()

This output show a point cloud constructed from a depth image.

### From voxels
This subsection intorduce a method to construct a point cloud from voxels. The subsection tutorial use following code:

In [36]:
import numpy as np
from tutlibs.constructing import voxel_to_point
from tutlibs.visualization import JupyterVisualizer as jv

In [38]:
voxels = np.load("../data/bunny_vox.npy")
point_cloud = voxel_to_point(voxels)
obj_points = jv.point(point_cloud, point_size=0.5)
jv.display([obj_points])

Output()

This output show a point cloud constructed from voxels.

## To voxels
### From a point cloud
This subsection intorduce a method to construct voxels from a point cloud. The subsection tutorial use following code:

In [32]:
import numpy as np
from tutlibs.io import Points
from tutlibs.constructing import point_to_voxel
from tutlibs.visualization import JupyterVisualizer as jv

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [33]:
coords, _, _ = Points.read("../data/bunny_pc.ply")
voxels = point_to_voxel(coords, 0.05)
obj_voxels = jv.voxel(voxels)
jv.display([obj_voxels])

Output()

This output show voxels constructed from a point cloud.

## To triangle meshes
### From voxels
This subsection intorduce a method to construct triangle meshes from a point cloud. The subsection tutorial use following code:

In [52]:
from tutlibs.constructing import marching_cubes

import numpy as np
from tutlibs.transformation import Transformation as tr
from tutlibs.visualization import JupyterVisualizer as jv

In [72]:
voxels = np.load("../data/bunny_vox.npy")
N, _, _ = voxels.shape

new_voxels = np.zeros((N+2, N+2, N+2))
new_voxels[1:N+1, 1:N+1, 1:N+1] = voxels

# TODO tell surface
vertices, triangles = marching_cubes(new_voxels)

vertices = tr.translation(vertices, np.array([N-1, 0-1, 0-1]))
obj_voxels = jv.voxel(voxels)
obj_meshes = jv.mesh(vertices, triangles)
jv.display([obj_voxels, obj_meshes])

/home/obarads/anaconda3/envs/pct/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/home/obarads/anaconda3/envs/pct/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()